## I need to answer several questions. The answers to those questions must be supported by data and analytics. These are the questions:

### 1. Which type of complaint should the Department of Housing Preservation and Development of New York City focus on first?
### 2. Should the Department of Housing Preservation and Development of New York City focus on any particular set of boroughs, ZIP codes, or street (where the complaints are severe) for the specific type of complaints you identified in response to Question 1?

### 3. Does the Complaint Type that you identified in response to question 1 have an obvious relationship with any particular characteristic or characteristics of the houses or buildings?
### 4. Can a predictive model be built for a future prediction of the possibility of complaints of the type that you have identified in response to question 1?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df= pd.read_csv('input/311_Service_Requests_from_2010_to_Present_min.csv')
print(df.shape)
df.head()

(6019843, 16)


,Unnamed: 0,Unique Key,Created Date,Closed Date,Complaint Type,Location Type,Incident Zip,Incident Address,Street Name,Address Type,City,Status,Resolution Description,Borough,Latitude,Longitude
0,0,45531130,02/02/2020 06:09:17 AM,NaN,HEAT/HOT WATER,RESIDENTIAL BUILDING,10019.0,426 WEST 52 STREET,WEST 52 STREET,ADDRESS,NEW YORK,Open,The following complaint conditions are still o...,MANHATTAN,40.765132,-73.988993
1,1,45529784,02/02/2020 02:15:24 PM,NaN,UNSANITARY CONDITION,RESIDENTIAL BUILDING,11204.0,1751 67 STREET,67 STREET,ADDRESS,BROOKLYN,Open,The following complaint conditions are still o...,BROOKLYN,40.618484,-73.992673
2,2,45527528,02/02/2020 02:27:41 AM,NaN,HEAT/HOT WATER,RESIDENTIAL BUILDING,11372.0,87-15 37 AVENUE,37 AVENUE,ADDRESS,Jackson Heights,Open,The following complaint conditions are still o...,QUEENS,40.750269,-73.879432
3,3,45530329,02/02/2020 12:13:18 PM,NaN,HEAT/HOT WATER,RESIDENTIAL BUILDING,10458.0,2405 SOUTHERN BOULEVARD,SOUTHERN BOULEVARD,ADDRESS,BRONX,Open,The following complaint conditions are still o...,BRONX,40.853773,-73.881558
4,4,45528814,02/02/2020 01:59:44 PM,NaN,APPLIANCE,RESIDENTIAL BUILDING,11209.0,223 78 STREET,78 STREET,ADDRESS,BROOKLYN,Open,The following complaint conditions are still o...,BROOKLYN,40.629745,-74.030533


In [3]:
df.columns

Index(['Unnamed: 0', 'Unique Key', 'Created Date', 'Closed Date',
       'Complaint Type', 'Location Type', 'Incident Zip', 'Incident Address',
       'Street Name', 'Address Type', 'City', 'Status',
       'Resolution Description', 'Borough', 'Latitude', 'Longitude'],
      dtype='object')

### Let's get rid of all the unnecessary fields

In [4]:
df=df[['Unique Key', 'Created Date', 'Closed Date',
       'Complaint Type', 'Location Type', 'Incident Zip',
       'Incident Address', 'Street Name','Address Type',
       'City', 'Status', 'Resolution Description','Borough',
       'Latitude', 'Longitude']]
df.head()

,Unique Key,Created Date,Closed Date,Complaint Type,Location Type,Incident Zip,Incident Address,Street Name,Address Type,City,Status,Resolution Description,Borough,Latitude,Longitude
0,45531130,02/02/2020 06:09:17 AM,NaN,HEAT/HOT WATER,RESIDENTIAL BUILDING,10019.0,426 WEST 52 STREET,WEST 52 STREET,ADDRESS,NEW YORK,Open,The following complaint conditions are still o...,MANHATTAN,40.765132,-73.988993
1,45529784,02/02/2020 02:15:24 PM,NaN,UNSANITARY CONDITION,RESIDENTIAL BUILDING,11204.0,1751 67 STREET,67 STREET,ADDRESS,BROOKLYN,Open,The following complaint conditions are still o...,BROOKLYN,40.618484,-73.992673
2,45527528,02/02/2020 02:27:41 AM,NaN,HEAT/HOT WATER,RESIDENTIAL BUILDING,11372.0,87-15 37 AVENUE,37 AVENUE,ADDRESS,Jackson Heights,Open,The following complaint conditions are still o...,QUEENS,40.750269,-73.879432
3,45530329,02/02/2020 12:13:18 PM,NaN,HEAT/HOT WATER,RESIDENTIAL BUILDING,10458.0,2405 SOUTHERN BOULEVARD,SOUTHERN BOULEVARD,ADDRESS,BRONX,Open,The following complaint conditions are still o...,BRONX,40.853773,-73.881558
4,45528814,02/02/2020 01:59:44 PM,NaN,APPLIANCE,RESIDENTIAL BUILDING,11209.0,223 78 STREET,78 STREET,ADDRESS,BROOKLYN,Open,The following complaint conditions are still o...,BROOKLYN,40.629745,-74.030533


In [5]:
df[['Address Type']].describe()

,Address Type
count,5935078
unique,1
top,ADDRESS
freq,5935078


### The field "Address Type" seems to have only one value. It's not useful information. We will LET IT GO

In [6]:
df=df.drop(columns=['Address Type'])

In [7]:
df_comp=df.groupby('Complaint Type')[["Unique Key"]].count()
df_comp.sort_values('Unique Key',ascending=False).head()

,Unique Key
Complaint Type,
HEAT/HOT WATER,1261574
HEATING,887850
PLUMBING,711130
GENERAL CONSTRUCTION,500863
UNSANITARY CONDITION,451643


In [8]:
df_comp=df_comp[df_comp['Unique Key']>80000].sort_values(by='Unique Key',ascending=False)
df_comp.columns=['No of complaints']
df_comp.T

Complaint Type,HEAT/HOT WATER,HEATING,PLUMBING,GENERAL CONSTRUCTION,UNSANITARY CONDITION,PAINT - PLASTER,PAINT/PLASTER,ELECTRIC,NONCONST,DOOR/WINDOW,WATER LEAK,GENERAL,FLOORING/STAIRS,APPLIANCE
No of complaints,1261574,887850,711130,500863,451643,361257,346438,307310,260890,205278,193631,151308,137402,112831


# ***It seems like the highest complaints are for HEAT or HOT WATER!!***

### We are only going to focus on the most frequent occuring problems

In [9]:
print(df.shape)
df=df[df['Complaint Type'].isin(df_comp.index)]
print(df.shape)
df.head()

(6019843, 14)
(5889405, 14)


,Unique Key,Created Date,Closed Date,Complaint Type,Location Type,Incident Zip,Incident Address,Street Name,City,Status,Resolution Description,Borough,Latitude,Longitude
0,45531130,02/02/2020 06:09:17 AM,NaN,HEAT/HOT WATER,RESIDENTIAL BUILDING,10019.0,426 WEST 52 STREET,WEST 52 STREET,NEW YORK,Open,The following complaint conditions are still o...,MANHATTAN,40.765132,-73.988993
1,45529784,02/02/2020 02:15:24 PM,NaN,UNSANITARY CONDITION,RESIDENTIAL BUILDING,11204.0,1751 67 STREET,67 STREET,BROOKLYN,Open,The following complaint conditions are still o...,BROOKLYN,40.618484,-73.992673
2,45527528,02/02/2020 02:27:41 AM,NaN,HEAT/HOT WATER,RESIDENTIAL BUILDING,11372.0,87-15 37 AVENUE,37 AVENUE,Jackson Heights,Open,The following complaint conditions are still o...,QUEENS,40.750269,-73.879432
3,45530329,02/02/2020 12:13:18 PM,NaN,HEAT/HOT WATER,RESIDENTIAL BUILDING,10458.0,2405 SOUTHERN BOULEVARD,SOUTHERN BOULEVARD,BRONX,Open,The following complaint conditions are still o...,BRONX,40.853773,-73.881558
4,45528814,02/02/2020 01:59:44 PM,NaN,APPLIANCE,RESIDENTIAL BUILDING,11209.0,223 78 STREET,78 STREET,BROOKLYN,Open,The following complaint conditions are still o...,BROOKLYN,40.629745,-74.030533


### Which area had the largest number of complaints?

In [10]:
df_bor= df.groupby('Borough')[['Unique Key']].count().sort_values('Unique Key',ascending=False)
df_bor

,Unique Key
Borough,
BROOKLYN,1712511
BRONX,1597799
MANHATTAN,1040696
Unspecified,818831
QUEENS,633840
STATEN ISLAND,85728


## It seems like Brooklyn has the highest number of complaints. But BRONX is also very close. There are also some unspecified entries. We will have to find what borough those zip numbers belong to.

In [11]:
df_pluto = pd.read_csv('/kaggle/input/nyc-pluto/pluto_20v2.csv')
df_pluto.head()

FileNotFoundError: [Errno 2] File /kaggle/input/nyc-pluto/pluto_20v2.csv does not exist: '/kaggle/input/nyc-pluto/pluto_20v2.csv'

In [ ]:
df_zip=df.groupby('Incident Zip')[['Borough']].agg(lambda x:x.value_counts().index[0])
df_zip.head()

In [ ]:
for i,j in zip(df[df['Borough']=='Unspecified'].index,df[df['Borough']=='Unspecified']['Incident Zip']):
    if np.isnan(j):
        continue
    df.at[i,'Borough']=df_zip.at[j,'Borough']
    #print(type(j))
    


In [ ]:
df[df['Borough']=='Unspecified'].head()

In [ ]:
df.groupby('Borough')[['Unique Key']].count().sort_values('Unique Key',ascending=False)

## We still have some Unspecified values. But they are because their zip wasn't given. Since their number is now less by multiple factors of ten, we will ignore the rest.

### Let's see which address has the most complaints

In [ ]:
df.groupby('Incident Address')[['Unique Key']].count().sort_values('Unique Key',ascending=False)

In [ ]:
df[df['Incident Address']=='34 ARDEN STREET'][['Incident Address','Incident Zip','Borough']].head(1)


### The address where most number of complaints came from is 
# 34 ARDEN STREET, MANHATTAN 10040

In [ ]:
df.groupby('Incident Zip')[['Unique Key']].count().sort_values('Unique Key',ascending=False).head()

## The zipcode where the most number of complaints came from is 
# 11226

In [ ]:
df.groupby('Status')[['Unique Key']].count().sort_values('Unique Key',ascending=False).head()

In [ ]:
df_pluto=df_pluto[['address','bldgarea','bldgdepth','builtfar','commfar','facilfar','lot','lotarea',
                   'lotdepth','numbldgs','numfloors','officearea','resarea','residfar','retailarea',
                   'yearbuilt','yearalter1','zipcode','ycoord','xcoord']]
df_pluto.shape

In [ ]:
df_pluto['bldgage']=2020-df_pluto['yearbuilt']
df_pluto.head()

In [ ]:
df_comp_count=df.groupby('Incident Address')[['Incident Address']].count()

In [ ]:
df_comp_count.columns=['count of complaints']
df_comp_count['address']=df_comp_count.index
df_comp_count.head()

In [ ]:
#df_comp_count.index=None
df_comp_count.reset_index(drop=True,inplace=True)
df_comp_count.head()

In [ ]:
df_corr = pd.merge(df_comp_count,df_pluto,on='address')
df_corr.head()

In [ ]:
print(df_comp_count.shape)
print(df_pluto.shape)
df_corr.shape

In [ ]:
df_corr.corr()